In [1]:
#シミュレーション
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm

In [5]:
"""
def rk4(func, t, h, x, *p)
4次のルンゲ・クッタ法を一回分計算する関数
    引数リスト
    func:導関数
    t：現在時刻を表す変数
    h：刻み幅
    x：出力変数（求めたい値）
    *p:引数の数が可変する事に対応する、その他の必要変数
※この関数では時刻は更新されないため、これとは別に時間更新をする必要があります。

導関数の書き方
def func(t, x, *state):
    func:自分で好きな関数名をつけられます
    t:時刻変数(変数の文字はtで無くても良い) 
    x:出力変数(変数の文字はxで無くても良い)
    *state:その他の必要変数(引数の数は可変可能))
#関数サンプル
def vdot(t, x, *state):
    s1=state[0]
    s2=state[1]
    return t+x+s1+s2
    
"""

def rk4(func, t, h, x, *p):
    k1=h*func(t, x, *p)
    k2=h*func(t+0.5*h, x+0.5*k1, *p)
    k3=h*func(t+0.5*h, x+0.5*k2, *p) 
    k4=h*func(t+h, x+k3, *p)
    x=x+(k1 + 2*k2 + 2*k3 + k4)/6
    return x


In [ ]:
#マルチコプタサンプルの数値例計算　Ver 0.0

###重力加速度
grav=9.80665

###モータのパラメータ
Jm=3.4e-6
Lm=3.7e-6
Rm=0.12
Km=3.28e-3
Dm=0.0
Qf=0.0

###プロペラのパラメータ
Ct=8.3e-7
Cq=3.0e-8

###各パーツの質量(kg)
mm=0.045 #モータ+プロペラ 0.045
mf=0.19 #フレーム
ml=0.02 #脚
mb=0.26 #電池

###各パーツの大きさ(m)
##モータ
Dim=0.038 #直径
rm=Dim/2 #半径
hm=0.016 #高さ

##フレーム
ef=0.292 #長辺の長さ
af=0.127 #１本のアームの根本からの長さ
aw=ef-2*af #アームの幅
df=(ef-af)/np.sqrt(2) #フレームの中心とフレームの空き空間の中心距離

##脚
Dl=0.03 #脚の直径
rl=Dl/2 #脚の半径
hl=0.036 #脚の高さ

##電池
hb=0.016 #電池高さ
wb=0.05 #電池幅
lb=0.15 #電池長さ

###各パーツの位置関係
##x軸
mlx=0.0 #モータ＋プロペラと脚の距離（x軸）
fbx=0.0 #フレームと電池の距離（x軸）
flx=0.18/2 #フレームと脚の距離（x軸）
llx=0.18 #脚の間隔
mmx=0.18 #モータの間隔
mfx=0.18/2
##y軸
mly=0.0 #モータ＋プロペラと脚の距離（y軸）
fby=0.0 #フレームと電池の距離（y軸）
fly=0.18/2 #フレームと脚の距離（y軸）
lly=0.18 #脚の間隔
mmy=0.18 #モータの間隔
mfy=0.18/2
##z軸
mlz=0.046 #モータ＋プロペラと脚の距離（z軸）
fbz=0.018 #フレームと電池の距離（z軸）
flz=0.028 #フレームと脚の距離（z軸）

###各パーツの機体重心を原点とした座標
##x軸
mlFR=2*ml
mlRR=2*ml
mmFR=2*mm
mmRR=2*mm
Ax=np.matrix([[2*mlFR, 2*mlRR, mb, mf, 2*mmFR, 2*mmRR],\
              [1,-1, 0, 0, 0, 0],\
              [0, 0, 0, 0, 1,-1],\
              [0, 0, 1,-1, 0, 0],\
              [1, 0, 0,-1, 0, 0],\
              [0, 0, 0,-1, 1, 0]])
Bx=np.matrix([[0],[llx],[mmx],[fbx],[mfx],[flx]])
Xa=np.linalg.inv(Ax)*Bx
lFRx=lFLx=Xa[0,0]
lRRx=lRlx=Xa[1,0]
bx=Xa[2,0]
fx=Xa[3,0]
mFRx=mFLx=Xa[4,0]
mRRx=mRLx=Xa[5,0]

##y軸
mlRR=2*ml
mlRL=2*ml
mmRR=2*mm
mmRL=2*mm
Ay=np.matrix([[2*mlRR, 2*mlRL, mb, mf, 2*mmRR, 2*mmRL],\
              [1,-1, 0, 0, 0, 0],\
              [0, 0, 0, 0, 1,-1],\
              [0, 0, 1,-1, 0, 0],\
              [1, 0, 0,-1, 0, 0],\
              [0, 0, 0,-1, 1, 0]])
By=np.matrix([[0],[lly],[mmy],[fby],[mfy],[fly]])
Ya=np.linalg.inv(Ay)*By
lFRy=lRRy=Ya[0,0]
lFLy=lRLy=Ya[1,0]
by=Ya[2,0]
fy=Ya[3,0]
mFRy=mRRy=Ya[4,0]
mFLy=mRLy=Ya[5,0]

##z軸
Az=np.matrix([[4*ml, mb, mf, 4*mm],\
              [-1, 0, 0, 1],\
              [ 0,-1, 1, 0],\
              [-1, 0, 1, 0]])
Bz=np.matrix([[0],[mlz],[fbz],[flz]])
Za=np.linalg.inv(Az)*Bz
lz=Za[0,0]
bz=Za[1,0]
fz=Za[2,0]
mz=Za[3,0]


###各パーツの慣性モーメント
##モータ
Imx=Imy=0.25*mm*rm**2+mm*hm**2/12
Imz=0.5*mm*rm**2

##フレーム
#計算準備
sf=4*(af*aw)+aw**2 #フレームの上面から見た面積
se=af**2 #空白部分の面積
sl=aw*ef #長い長方形の面積
sa=aw*af #アームの面積
mfs=mf*ef**2/sf #空白部分が満たされていたときの質量
mfe=mf*se/sf #空白部分の質量
mfl=mf*sl/sf #長い長方形の質量
ma=mf*sa/sf #アームの質量
#ここから慣性モーメント
Icg= mfe*af**2/6 + mfe*df**2 #空き空間に正方形の物体が有ったとしたときの慣性モーメント
Ifx=Ify=mfl*(ef**2+aw**2)/12+2*ma*(af**2+aw**2)/12
Ifz=mfs*ef**2/6-4*Icg

##脚
Ilx=Ily=0.5*ml*rl**2
Ilz= 0.25*ml*rl**2+ ml*hl**2/12

##電池
Ibx=mb*(wb**2+hb**2)/12
Iby=mb*(lb**2+hb**2)/12
Ibz=mb*(wb**2+lb**2)/12

###各パーツの重心からの距離
##xz&yz平面
dlxz=dlyz=np.sqrt(lFRx**2 + lz**2)
dbxz=dbyz=np.sqrt(  bx**2 + bz**2)
dfxz=dfyz=np.sqrt(  fx**2 + fz**2)
dmxz=dmyz=np.sqrt(mFRx**2 + mz**2)
#print('dlxz={:10.2e} \ndbxz={:10.2e} \ndfxz={:10.2e} \ndmxz={:10.2e}\n'.format(dlxz,dbxz,dfxz,dmxz))

##xy平面
dlxy=np.sqrt(lFRx**2 + lFRy**2)
dbxy=np.sqrt(  bx**2 + by**2)
dfxy=np.sqrt(  fx**2 + fy**2)
dmxy=np.sqrt(mFRx**2 + mFRy**2)
#print('dlxy={:10.2e} \ndbxy={:10.2e} \ndfxy={:10.2e} \ndmxy={:10.2e}\n'.format(dlxy,dbxy,dfxy,dmxy))


###マルチコプタの慣性モーメント
##x軸周り　yz平面
Ix=(4*(Imx + mm*dmyz**2)+\
       Ifx + mf*dfyz**2 +\
       Ibx + mb*dbyz**2 +\
       4*(Ilx + ml*dlyz**2))
##y軸周り　xz平面
Iy=(4*(Imy + mm*dmxz**2)+\
       Ify + mf*dfxz**2 +\
       Iby + mb*dbxz**2 +\
       4*(Ily + ml*dlxz**2))
##z軸周り　xy平面
Iz=(4*(Imz + mm*dmxy**2)+\
    Ifz + mf*dfxy**2 +\
    Ibz + mb*dbxy**2 +\
    4*(Ilz + ml*dlxy**2))

###マルチコプタ質量
Mass=mm*4+mf+ml*4+mb

### 伝達関数パラメータ
omega0=0.5*np.sqrt(Mass*grav/Ct)
e0=Rm*((Dm + Km**2/Rm)*omega0 + Cq*omega0**2 + Qf)/Km
taum=Jm*Rm/(Dm*Rm+Km**2+2*Rm*Cq*omega0)
Kw=2*Ct*omega0*Km/(Dm*Rm+Km**2+2*Rm*Cq*omega0)/Mass
Kroll=mmx*Ct*omega0*Km/(Dm*Rm+Km**2+2*Rm*Cq*omega0)/Ix
Kpitch=mmx*Ct*omega0*Km/(Dm*Rm+Km**2+2*Rm*Cq*omega0)/Iy
Kyaw=2*Cq*omega0*Km/(Dm*Rm+Km**2+2*Rm*Cq*omega0)/Iz


###結果表示
print('### Mass ###')
print('{:10.2e}'.format(Mass))

print('\n### Equilibrium value ###')
print('omega0={:10.2e}'.format(omega0))
print('e0={:10.2e}'.format(e0))

print('\n### Transfer Function Parameter ###')
print('tau={:10.2e}'.format(taum))
print('Kw={:10.2e}'.format(Kw))
print('Kroll(Kp)={:10.2e}'.format(Kroll))
print('Kpitch(Kq)={:10.2e}'.format(Kpitch))
print('Kyaw(Kr)={:10.2e}'.format(Kyaw))

print('\n### Inertia　tensor  ###\n')
print('Motor+Prop')
print('Imx={:10.2e}'.format(Imx))
print('Imy={:10.2e}'.format(Imy))
print('Imz={:10.2e}'.format(Imz))
print('\nFrame')
print('Ifx={:10.2e}'.format(Ifx))
print('Ify={:10.2e}'.format(Ify))
print('Ifz={:10.2e}'.format(Ifz))
print('\nLeg')
print('Ilx={:10.2e}'.format(Ilx))
print('Ily={:10.2e}'.format(Ily))
print('Ilz={:10.2e}'.format(Ilz))
print('\nBattery')
print('Ibx={:10.2e}'.format(Ibx))
print('Iby={:10.2e}'.format(Iby))
print('Ibz={:10.2e}'.format(Ibz))

print('\n## Multi Copter ##')
print('Ix={:10.2e}'.format(Ix))
print('Iy={:10.2e}'.format(Iy))
print('Iz={:10.2e}'.format(Iz))

print('\n### Position from Center of Gravity ###')
print('\nMotor Position')
print('Front Right Motor:({:10.2e},{:10.2e},{:10.2e})'.format(mFRx,mFRy,mz))
print('Front Left  Motor:({:10.2e},{:10.2e},{:10.2e})'.format(mFLx,mFLy,mz))
print('Rear  Right Motor:({:10.2e},{:10.2e},{:10.2e})'.format(mRRx,mRRy,mz))
print('Rear  Left  Motor:({:10.2e},{:10.2e},{:10.2e})'.format(mRLx,mRLy,mz))

print('\nFrame Position')
print('({:10.2e},{:10.2e},{:10.2e})'.format(fx,fy,fz))

print('\nBattery Position')
print('({:10.2e},{:10.2e},{:10.2e})'.format(bx,by,bz))

print('\nLeg Position')
print('Front Right Leg:({:10.2e},{:10.2e},{:10.2e})'.format(lFRx,lFRy,lz))
print('Front Left  Leg:({:10.2e},{:10.2e},{:10.2e})'.format(lFLx,lFLy,lz))
print('Rear  Right Leg:({:10.2e},{:10.2e},{:10.2e})'.format(lRRx,lRRy,lz))
print('Rear  Left  Leg:({:10.2e},{:10.2e},{:10.2e})'.format(lRlx,lRLy,lz))


In [1]:
#マルチコプタの運動
class multicopter:
    
    grav=9.80665
    
    def __init__(self, M, Jmot, D, Qf, R, K, Ct, Cq, Ix, Iy, Iz, l):
        self.M=M
        self.J=Jmot
        self.D=D
        self.Qf=Qf
        self.R=R
        self.K=K
        self.Ct=Ct
        self.Cq=Cq
        self.Ix=Ix
        self.Iy=Iy
        self.Iz=Iz
        self.l=l
        
    #導関数
    def xdot(self, t, x, angle, u):
        u=x[0]
        v=x[1]
        w=x[2]
        p=x[3]
        q=x[4]
        r=x[5]
        omega_fr=x[6]
        omega_fl=x[7]
        omega_rr=x[8]
        omega_rl=x[9]
        
        phi=angle[0]
        theta=angle[1]
        psi=angle[2]
        
        efr=u[0]
        efl=u[1]
        err=u[2]
        erl=u[3]
        
        Tfr=Ct*omega_fr**2
        Tfl=Ct*omega_fl**2
        Trr=Ct*omega_rr**2
        Trl=Ct*omega_rl**2        
        
        Qfr=Cq*omega_fr**2
        Qfl=Cq*omega_fl**2
        Qrr=Cq*omega_rr**2
        Qrl=Cq*omega_rl**2
        
        udot=-grav*np.sin(theta) -q*w +r*v
        vdot= grav*np.cos(theta)*sin(phi) -r*u +p*w
        wdot= grav*np.cos(theta)*cos(phi) -p*v +p*u +\
                -(Tfr + Tfl + Trr + Trl)/m
        
        pdot=(0.5*self.l*(Tfr + Trr - Tfl - Trl ) - (Iz -Iy)*q*r)/Ix
        qdot=(0.5*self.l*(Trr + Trl - Tfr - Tfl ) - (Ix -Iz)*r*p)/Iy
        rdot=((Qfr + Qrl - Qfl - Qrr ) - (Iy -Ix)*p*q)/Iz
        
        omegadot_fr=(self.K/self.R*efr \
                    - (self.D + self.K**2/self.R)*omega_fr\
                    - self.Cq*omega_fr**2\
                    )/self.J
        omegadot_fl=(self.K/self.R*efl \
                    - (self.D + self.K**2/self.R)*omega_fl\
                    - self.Cq*omega_fl**2\
                    )/self.J
        omegadot_rr=(self.K/self.R*err \
                    - (self.D + self.K**2/self.R)*omega_rr\
                    - self.Cq*omega_rr**2\
                    )/self.J
        omegadot_rl=(self.K/self.R*erl \
                    - (self.D + self.K**2/self.R)*omega_rl\
                    - self.Cq*omega_rl**2\
                    )/self.J
        
        return (udot, vdot, wdot,\
                pdot, qdot, rdot,\
                omegadot_fr, omegadot_fl, omegadot_rr, omegadot_rl)
    

    